In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.pipeline import Pipeline
from sklearn import set_config; set_config(display = "diagram")
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import TransformerMixin
from sklearn.base import BaseEstimator
from sklearn.pipeline import FeatureUnion

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('final_le_wagon_project_data.csv')

In [3]:
df.head(5)

,sr_no,gender,category,grade_10,grade_12,graduation_score,graduation_type,admission_test_score,work_exp_months,shortlist_uni_1,...,shortlist_uni_2,admit_uni_2,shortlist_uni_3,admit_uni_3,shortlist_uni_4,admit_uni_4,shortlist_uni_5,admit_uni_5,shortlist_uni_6,admit_uni_6
0,1,Male,General,82.0,86.5,77.4,Engineer,92.43,30,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Female,General,95.0,95.6,85.4,Engineer,94.29,16,0,...,0,0,0,0,0,0,1,0,0,0
2,3,Female,General,93.1,91.6,79.2,Non-Engineer,92.52,4,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Male,General,91.8,89.7,80.7,Non-Engineer,96.57,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Female,General,96.0,86.2,92.4,Non-Engineer,91.00,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df = df.drop(['sr_no', 'admit_uni_1', 'admit_uni_2', 'admit_uni_3', 'admit_uni_4', 'admit_uni_5', 'admit_uni_6'], axis=1)
df = df.dropna().copy()

In [5]:
df.head(10)

,gender,category,grade_10,grade_12,graduation_score,graduation_type,admission_test_score,work_exp_months,shortlist_uni_1,shortlist_uni_2,shortlist_uni_3,shortlist_uni_4,shortlist_uni_5,shortlist_uni_6
0,Male,General,82.0,86.5,77.4,Engineer,92.43,30,0,0,0,0,0,0
1,Female,General,95.0,95.6,85.4,Engineer,94.29,16,0,0,0,0,1,0
2,Female,General,93.1,91.6,79.2,Non-Engineer,92.52,4,0,0,0,0,0,0
3,Male,General,91.8,89.7,80.7,Non-Engineer,96.57,0,0,0,0,0,0,0
4,Female,General,96.0,86.2,92.4,Non-Engineer,91.00,0,0,0,0,0,0,0
5,Male,General,87.4,92.6,79.9,Engineer,99.63,21,0,0,1,0,0,0
6,Male,NC_OBC,99.3,89.7,73.6,Engineer,95.01,25,0,1,0,1,1,1
7,Male,General,95.0,86.8,71.1,Engineer,99.26,26,0,0,0,0,0,0
8,Male,General,87.4,79.6,70.1,Engineer,99.22,28,0,0,0,0,0,1
9,Female,NC_OBC,85.5,84.3,70.9,Non-Engineer,91.50,14,0,0,0,0,1,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3940 entries, 0 to 3939
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gender                3940 non-null   object 
 1   category              3940 non-null   object 
 2   grade_10              3940 non-null   float64
 3   grade_12              3940 non-null   float64
 4   graduation_score      3940 non-null   float64
 5   graduation_type       3940 non-null   object 
 6   admission_test_score  3940 non-null   float64
 7   work_exp_months       3940 non-null   int64  
 8   shortlist_uni_1       3940 non-null   int64  
 9   shortlist_uni_2       3940 non-null   int64  
 10  shortlist_uni_3       3940 non-null   int64  
 11  shortlist_uni_4       3940 non-null   int64  
 12  shortlist_uni_5       3940 non-null   int64  
 13  shortlist_uni_6       3940 non-null   int64  
dtypes: float64(4), int64(7), object(3)
memory usage: 431.1+ KB


In [7]:
final_preprocessor = ColumnTransformer([
    ('num_encoder', RobustScaler(), make_column_selector(dtype_exclude="object")),
    ('cat_encoder', OneHotEncoder(handle_unknown='ignore', drop = 'first', sparse=False), make_column_selector(dtype_include="object"))])

In [8]:
final_preprocessor

ColumnTransformer(transformers=[('num_encoder', RobustScaler(),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x11abaa4a0>),
                                ('cat_encoder',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore',
                                               sparse=False),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x11abaa440>)])

In [9]:
X = df.drop(columns=["shortlist_uni_1", "shortlist_uni_2", "shortlist_uni_3",
                    "shortlist_uni_4", "shortlist_uni_5", "shortlist_uni_6" ])

y1 = df["shortlist_uni_1"]
y2 = df["shortlist_uni_2"]
y3 = df["shortlist_uni_3"]
y4 = df["shortlist_uni_4"]
y5 = df["shortlist_uni_5"]
y6 = df["shortlist_uni_6"]

In [10]:
df.head()

,gender,category,grade_10,grade_12,graduation_score,graduation_type,admission_test_score,work_exp_months,shortlist_uni_1,shortlist_uni_2,shortlist_uni_3,shortlist_uni_4,shortlist_uni_5,shortlist_uni_6
0,Male,General,82.0,86.5,77.4,Engineer,92.43,30,0,0,0,0,0,0
1,Female,General,95.0,95.6,85.4,Engineer,94.29,16,0,0,0,0,1,0
2,Female,General,93.1,91.6,79.2,Non-Engineer,92.52,4,0,0,0,0,0,0
3,Male,General,91.8,89.7,80.7,Non-Engineer,96.57,0,0,0,0,0,0,0
4,Female,General,96.0,86.2,92.4,Non-Engineer,91.00,0,0,0,0,0,0,0


In [11]:
pd.DataFrame(final_preprocessor.fit_transform(X)).head()

,0,1,2,3,4,5,6,7,8,9,10
0,-1.368421,-0.250000,-0.008197,-0.321195,1.666667,1.0,1.0,0.0,0.0,0.0,0.0
1,0.342105,0.592593,0.647541,-0.089636,0.888889,0.0,1.0,0.0,0.0,0.0,0.0
2,0.092105,0.222222,0.139344,-0.309991,0.222222,0.0,1.0,0.0,0.0,0.0,1.0
3,-0.078947,0.046296,0.262295,0.194211,0.000000,1.0,1.0,0.0,0.0,0.0,1.0
4,0.473684,-0.277778,1.221311,-0.499222,0.000000,0.0,1.0,0.0,0.0,0.0,1.0


In [12]:
# Chain the final_preprocessor pipeline with a logistic regression model
pipe1 = Pipeline([
    ('preprocessing', final_preprocessor),
    ('logistic_regression', LogisticRegression())
])

pipe1

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_encoder', RobustScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11abaa4a0>),
                                                 ('cat_encoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11abaa440>)])),
                ('logistic_regression', LogisticRegression())])

In [13]:
pipe1.fit(X, y1)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_encoder', RobustScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11abaa4a0>),
                                                 ('cat_encoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11abaa440>)])),
                ('logistic_regression', LogisticRegression())])

In [14]:
pipe1['preprocessing'].transformers_

[('num_encoder',
  RobustScaler(),
  ['grade_10',
   'grade_12',
   'graduation_score',
   'admission_test_score',
   'work_exp_months']),
 ('cat_encoder',
  OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False,
                sparse_output=False),
  ['gender', 'category', 'graduation_type'])]

In [15]:
col_num = pipe1['preprocessing'].transformers_[0][2]
col_num

['grade_10',
 'grade_12',
 'graduation_score',
 'admission_test_score',
 'work_exp_months']

In [16]:
pipe1['preprocessing'].transformers_[1][1].get_feature_names_out()

array(['gender_Male', 'category_General', 'category_NC_OBC',
       'category_SC', 'category_ST', 'graduation_type_Non-Engineer'],
      dtype=object)

In [17]:
col_cat = list(pipe1['preprocessing'].transformers_[1][1].get_feature_names_out())
col_cat

['gender_Male',
 'category_General',
 'category_NC_OBC',
 'category_SC',
 'category_ST',
 'graduation_type_Non-Engineer']

In [18]:
import itertools
col_preproc = list(itertools.chain(col_num, col_cat))

In [19]:
len(col_preproc)

11

In [20]:
pd.DataFrame(data=pipe1['preprocessing'].fit_transform(X), columns=col_preproc).head()

,grade_10,grade_12,graduation_score,admission_test_score,work_exp_months,gender_Male,category_General,category_NC_OBC,category_SC,category_ST,graduation_type_Non-Engineer
0,-1.368421,-0.250000,-0.008197,-0.321195,1.666667,1.0,1.0,0.0,0.0,0.0,0.0
1,0.342105,0.592593,0.647541,-0.089636,0.888889,0.0,1.0,0.0,0.0,0.0,0.0
2,0.092105,0.222222,0.139344,-0.309991,0.222222,0.0,1.0,0.0,0.0,0.0,1.0
3,-0.078947,0.046296,0.262295,0.194211,0.000000,1.0,1.0,0.0,0.0,0.0,1.0
4,0.473684,-0.277778,1.221311,-0.499222,0.000000,0.0,1.0,0.0,0.0,0.0,1.0


In [30]:
scores = cross_val_score(pipe1, X, y1, cv=5, scoring="accuracy")
score = scores.mean()

print(scores)
print(scores.mean())

[0.90482234 0.9035533  0.9035533  0.90228426 0.9035533 ]
0.9035532994923858


In [24]:
import pickle

# Export pipeline as pickle file
with open("pipeline1.pkl", "wb") as file:
    pickle.dump(pipe1, file)

In [25]:
# Load pipeline from pickle file
my_pipeline1 = pickle.load(open("pipeline1.pkl", "rb"))

my_pipeline1

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_encoder', RobustScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11ae19a50>),
                                                 ('cat_encoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11ae19de0>)])),
                ('logistic_regression', LogisticRegression())])

In [26]:
new_data = pd.read_csv("/Users/prasadsawant/Downloads/new_data.csv")

In [27]:
new_data

,gender,category,grade_10,grade_12,graduation_score,graduation_type,admission_test_score,work_exp_months
0,Male,General,89.20,75.50,74.12,Non-Engineer,98.28,6
1,Female,NC_OBC,93.67,91.45,87.23,Engineer,99.27,23
2,Female,ST,85.78,80.23,82.80,Non-Engineer,85.60,9


In [28]:
predicted_class = my_pipeline1.predict(new_data)
predicted_class

array([0, 0, 0])